# iBioFoundry #4 - colony PCR - OT2
author: Camillo Moschner | version: 1.2 | date: 26.01.2022 | license = 

# 1- Metadata

In [1]:
metadata = {
    'protocolName': 'iBF4_d2_cPCR',
    'author': 'Camillo Moschner <cm967@cam.ac.uk> / <camillo.moschner@gmail.com>',
    'description': 'Automated preparation of cPCR reactions',
    'apiLevel': '2.12',
    'Date': '27.01.2022',
    'pipette_configuration':{'left':'p20_single_gen2',
                             'right':'p300_single_gen2'}
    }

# 2- Import Statements

In [2]:
import numpy as np
import itertools
import pandas as pd
from IPython.display import display, clear_output, Audio, display, display_html
from copy import deepcopy
import time
from datetime import date
import math
from copy import deepcopy
import os
from pprint import pprint

from iBioFoundry_helper import *
from opentrons import protocol_api, execute
from opentrons import simulate

## cPCR Master Inputs
The following cell requires changing between experimental setups (and, for simplicity for the user, is supposed to be the only part of the protocol that requires any modification).

In [3]:
# establish whether you are performing a simulation (e.g. at the design stage) or are executing the script on the OT-2
script_mode = 'simulation' # 'simulation' or 'execution'
experiment_name = 'sTUFPlibrary'
audiofeedback = False # True if you want to to receive audio-notification of a finished process, else change to Falsev

# how many colonies per assembly to perform cPCR on
no_of_col_tested_per_sample = 3
# colony info as starting point
assemblies_info_dir = f"1_GGA{os.path.sep}assemblies_summary.csv" #assemblies_info_df
# resuspended colony info
resusp_colony_plate_format = 'opentrons_96_aluminumblock_biorad_wellplate_200ul' # opentrons_96_aluminumblock_generic_pcr_strip_200ul
resusp_col_plate_column_usage = 'all' # allows you to selectively only use 'odd'- or 'even'-numbered columns or 'all'; useful for 0.2 ml tube usage
# PBS & PCR MM reagent info
PCR_reagent_definition_dir = f"input4b_qcPCR_reagent_definitions.csv" 
reagents_rack_format  = 'opentrons_24_tuberack_eppendorf_1.5ml_safelock_snapcap'
reagent_pos_dic = {'PBS' : 'A3',
                   'MM' : 'B3'}
# cPCR tubes info
cPCR_plate_format = 'opentrons_96_aluminumblock_generic_pcr_strip_200ul'
cPCR_plate_column_usage = 'all'
# final reaction composition
resusp_col_for_cPCR_vol = 2.0 # units: µl

In [4]:
# establish sole Jupyter Notebook control 
if script_mode == 'execution':
    try:
        os.system("systemctl stop opentrons-robot-server")
        protocol = execute.get_protocol_api(metadata['apiLevel']);
    except:
        protocol = execute.get_protocol_api(metadata['apiLevel']);
    print("You are operating this script in exectution mode!\nYou should see the lights turn on and hear the gantry homing!")
    startNow()
elif script_mode == 'simulation':
    protocol = simulate.get_protocol_api(metadata['apiLevel']);
    print("You are operating this script in simulation mode!")
protocol.set_rail_lights(True)
protocol.home()

/Users/camillomoschner/.opentrons/robot_settings.json not found. Loading defaults
/Users/camillomoschner/.opentrons/deck_calibration.json not found. Loading defaults


You are operating this script in simulation mode!


# 3- Function Definitions

In [5]:
def smart_transfer_liquid(transfer_vol, aspiration_pos,dispensation_pos,
                          pick_up_tip=True, touch_tip_before=True,
                          cyclical_mix_before=None,mix_before_reps=2,mix_rate=0.5,
                          asp_bot_clearance=1.5,dispens_bot_clearance=1.5,
                          asp_rate=0.8,disp_rate=0.5,
                          cyclical_mix_after=None,mix_after=None,
                          dispens_top_offset=-2, blow_out=False, air_gap=False, touch_tip=True, touch_miniscus=None,
                          drop_tip=True):
    """Liquid transfer that automatically identifies which tip to use.
    """
    if (transfer_vol >= p20_left.min_volume) & (transfer_vol <= p20_left.max_volume):
        pipette_to_use = p20_left
    elif (transfer_vol > p300_right.min_volume):
        pipette_to_use = p300_right
    else:
        print(f"WARNING: transfer volume not manageable")
    if pick_up_tip==True:
        pipette_to_use.pick_up_tip()
    if cyclical_mix_before != None: # give tuple (repetitions, mix_volume, well_bottom_clearance)
        for i in np.arange(cyclical_mix_before[0]):
            pipette_to_use.aspirate(cyclical_mix_before[1],aspiration_pos.bottom(cyclical_mix_before[2]))
            pipette_to_use.dispense(cyclical_mix_before[1],aspiration_pos.bottom(cyclical_mix_before[3]))
    if mix_before_reps != 0:
        pipette_to_use.mix(mix_before_reps,transfer_vol*0.8,aspiration_pos.bottom(asp_bot_clearance),mix_rate)
    # aspiration
    pipette_to_use.aspirate(transfer_vol,aspiration_pos.bottom(asp_bot_clearance), asp_rate)
    pipette_to_use.move_to(aspiration_pos.top())
    if air_gap != False:
        pipette_to_use.air_gap(air_gap)
    protocol.delay(seconds=1)
    if touch_tip_before == True:
        pipette_to_use.touch_tip(v_offset=-2, speed=150)
    # dispensation
    pipette_to_use.dispense(transfer_vol,dispensation_pos.bottom(dispens_bot_clearance),disp_rate)
    if cyclical_mix_after != None: # give tuple (repetitions, mix_volume, asp- , disp-well_bottom_clearance)
        for i in np.arange(cyclical_mix_after[0]):
            pipette_to_use.aspirate(cyclical_mix_after[1],dispensation_pos.bottom(cyclical_mix_after[2]))
            pipette_to_use.dispense(cyclical_mix_after[1],dispensation_pos.bottom(cyclical_mix_after[3]))
    if mix_after != None:
        pipette_to_use.mix(mix_after[0],mix_after[1], dispensation_pos.bottom(mix_after[2]),mix_rate)
    pipette_to_use.move_to(dispensation_pos.top())
    protocol.delay(seconds=1)
    if blow_out == True:
        pipette_to_use.blow_out(dispensation_pos.top(dispens_top_offset))
    if touch_tip == True:
        pipette_to_use.touch_tip(v_offset=dispens_top_offset, speed=150)
    if touch_miniscus != None:
        protocol.max_speeds['Z'] = 10  # limit z axis to 50 mm/s to avoid solution adhereing to outer tip surface
        pipette_to_use.move_to(dispensation_pos.bottom(touch_miniscus))
        protocol.max_speeds['Z'] = None # reset z axis speed
    if drop_tip==True:
        pipette_to_use.drop_tip()
        
def smart_distribute_liquid(single_distr_vol, aspiration_pos,dispensation_pos_list,
                            asp_bot_clearance=1.1,dispens_bot_clearance=1.8,
                            asp_flow_rate = 1,disp_flow_rate = 1, new_disposal_volume=None,
                            touch_tip_bool=True,blow_out_bool=False):
    """
    Liquid distribution from one aspiration well into multiple destination wells that automatically identifies which pipette to use.
    """
    if (single_distr_vol >= p20_left.min_volume) & (single_distr_vol <= p20_left.max_volume):
        pipette_to_use = p20_left
    elif (single_distr_vol > p300_right.min_volume):
        pipette_to_use = p300_right
    # save original/default flow rates
    original_asp_flow_rate = pipette_to_use.flow_rate.aspirate
    original_disp_flow_rate = pipette_to_use.flow_rate.dispense
    # change flow rates if desired
    pipette_to_use.flow_rate.aspirate = original_asp_flow_rate *asp_flow_rate
    pipette_to_use.flow_rate.dispense = original_disp_flow_rate *disp_flow_rate
    # change well_bottom_clearances if desired
    pipette_to_use.well_bottom_clearance.aspirate = asp_bot_clearance
    pipette_to_use.well_bottom_clearance.dispense = dispens_bot_clearance
    # execute
    if new_disposal_volume != None:
        pipette_to_use.distribute(single_distr_vol, aspiration_pos,dispensation_pos_list, 
                                  touch_tip=touch_tip_bool,disposal_volume=new_disposal_volume,blow_out=blow_out_bool)
    else:
        pipette_to_use.distribute(single_distr_vol, aspiration_pos,dispensation_pos_list,
                                  touch_tip=touch_tip_bool,blow_out=blow_out_bool)
    # set well_bottom_clearances back to default
    pipette_to_use.well_bottom_clearance.dispense = 1
    pipette_to_use.well_bottom_clearance.aspirate = 1
    # set flow rates back to default
    pipette_to_use.flow_rate.aspirate = original_asp_flow_rate
    pipette_to_use.flow_rate.dispense = original_disp_flow_rate

---
# 4- Deck assignments

In [6]:
"""Deck creation"""
deck_slot_list =list(np.arange(1,12))
deck_slot_list.append('bin')
deck_slot_df = pd.DataFrame( np.flip(np.array(deck_slot_list).reshape(4,3), axis=0) )

## i. Labware
Resource: [Opentrons Labware Library Webpage](https://labware.opentrons.com/?_gl=1*1iyc1t4*_ga*MTU1MTM1NzU5MS4xNjE2OTczMzY5*_ga_GNSMNLW4RY*MTYzMDYxNzY1NS4yMC4wLjE2MzA2MTc2NTUuMA..&_ga=2.110416899.767557364.1630617656-1551357591.1616973369)
### a. Modules

In [7]:
temp_mod_name  = 'temperature module gen2'
temp_mod = protocol.load_module(temp_mod_name,'3')
deck_slot_df.iloc[3,2] = temp_mod_name

### b. Racks & Plates

In [8]:
tiprack20a = protocol.load_labware('opentrons_96_tiprack_20ul','11')
tiprack20b = protocol.load_labware('opentrons_96_tiprack_20ul','10')
tiprack300a = protocol.load_labware('opentrons_96_tiprack_300ul','7')
# tuberacks & plates
resusp_colony_plate = protocol.load_labware(resusp_colony_plate_format,'2') # opentrons_96_aluminumblock_biorad_wellplate_200ul
cPCR_plate = temp_mod.load_labware(cPCR_plate_format,'3') # opentrons_24_aluminumblock_nest_1.5ml_screwcap
reagents_rack = protocol.load_labware(reagents_rack_format,'1') #  biorad_96_wellplate_200ul_pcr
large_vol_rack = protocol.load_labware('opentrons_6_tuberack_falcon_50ml_conical','4') 

In [9]:
labware_list = [tiprack20a,tiprack20b, tiprack300a,
                resusp_colony_plate,cPCR_plate, reagents_rack,large_vol_rack]
_ =[update_labware(deck_slot_df, labware_item) for labware_item in labware_list]
# visually inspect deck
print("\nFinal OT-2 Deck Configuration:");(deck_slot_df)


Final OT-2 Deck Configuration:


,0,1,2
0,10-opentrons_96_tiprack_20ul,11-opentrons_96_tiprack_20ul,bin
1,7-opentrons_96_tiprack_300ul,8,9
2,4-opentrons_6_tuberack_falcon_50ml_conical,5,6
3,1-opentrons_24_tuberack_eppendorf_1.5ml_safelo...,2-opentrons_96_aluminumblock_biorad_wellplate_...,temperature module gen2


## ii. Pipettes

Resource: [Opentrons Pipettes API Webpage](https://docs.opentrons.com/v2/new_pipette.html)

In [10]:
p20_left = protocol.load_instrument('p20_single_gen2','left', tip_racks=[tiprack20a,tiprack20b])
p300_right = protocol.load_instrument('p300_single_gen2','right', tip_racks=[tiprack300a])

## iii. Assembly & PCR info integration

In [11]:
assemblies_colony_info_df = pd.read_csv(assemblies_info_dir)
assembly_no = len(assemblies_colony_info_df)
reagent_definitions_df = pd.read_csv(PCR_reagent_definition_dir)

### In-Notebook Calibration

In [12]:
#if in_jupyter_calibration == True:
#import opentrons
pipette_to_calibrate = p20_left # p300_right
labware_to_calibrate_to = resusp_colony_plate
labware_position_to_test = 'A1'

In [13]:
# step 0: pick up tip for the pipette you want to calibrate and move to the labware to be calibrated
pipette_to_calibrate.pick_up_tip()
pipette_to_calibrate.move_to(labware_to_calibrate_to[labware_position_to_test].top())

<InstrumentContext: p20_single_v2.0 in LEFT>

In [14]:
# step 1: set_offset based on identifying in which direction the tip needs to move
# (can be an iderative process as this cell immediately moves to the new position)
labware_to_calibrate_to.set_offset(x=(0.),y=(0.),z=(0.))
pipette_to_calibrate.move_to(labware_to_calibrate_to[labware_position_to_test].top())

<InstrumentContext: p20_single_v2.0 in LEFT>

In [15]:
# step 2: return tip (if you haven'crashed it
pipette_to_calibrate.return_tip()

<InstrumentContext: p20_single_v2.0 in LEFT>

---
# 5- LH Step Calculations 
## cPCR Allocation

In [16]:
# append new columns for each tested colony to master spreadsheet
for colony_no in [f"cPCR_no{cPCR_no}" for cPCR_no in range(1,no_of_col_tested_per_sample+1)]:
    assemblies_colony_info_df[colony_no] = [np.nan for i in range(assembly_no)]
assemblies_colony_info_df['sequence_verified'] = [np.nan for i in range(assembly_no)]
# create reagent rack object
reagents_rack_obj = Plate(identify_plate(reagents_rack_format, 'plate_format'))
reagents_rack_obj.layout = extend_fill_plate_df_with_list(reagents_rack_obj.layout,reagent_definitions_df.reagent.to_list(),
                                                            fill_start_position='A1', fill_first='columns')
for reagent_name in reagent_pos_dic.keys():
    reagents_rack_obj.layout.loc[convert_to_position_tuple(reagent_pos_dic[reagent_name])] = reagent_name
# identify all assemblies for which (visually-inspected) colonies have grown
assemblies_colony_info_df_w_colonies_df = assemblies_colony_info_df.loc[assemblies_colony_info_df.colonies.notnull()]
no_assemblies_tested = len(assemblies_colony_info_df_w_colonies_df)
well_avail_per_plate = len(Plate(identify_plate(resusp_colony_plate_format, 'plate_format'),only_columns=resusp_col_plate_column_usage).well_list)

total_no_cPCRs = no_of_col_tested_per_sample*no_assemblies_tested
no_required_plates = math.ceil(total_no_cPCRs/well_avail_per_plate)
if no_assemblies_tested != 0:
    print(f"{len(assemblies_colony_info_df_w_colonies_df)} out of {assembly_no} colonies are selected for cPCR screening, and {no_of_col_tested_per_sample} cPCRs will be prepared for each of them.\n  - total cPCRs being performed: {total_no_cPCRs}")
    print(f"  - wells_avail_per_plate: {well_avail_per_plate}\n  - no_required_plates: {no_required_plates}")
else:
    print(f"WARNING: You have not identified how many samples have correct colonies ready for cPCR!")
# create multi-well plates objects for each plate
resusp_colony_plate_objs = [Plate(identify_plate(resusp_colony_plate_format, 'plate_format'),name=f"ResCol_Plate_{x}",only_columns=resusp_col_plate_column_usage) for x in range(no_required_plates)]
cPCR_plate_objs = [Plate(identify_plate(cPCR_plate_format, 'plate_format'),name=f"cPCR_Plate_{x}",only_columns=cPCR_plate_column_usage) for x in range(no_required_plates)]
# create a list of colony-identified assemblies, each multiplied by the number of cPCRs to perform
complete_cPCR_list_for_cPCR = [f"{assembly}_{col}"
                               for assembly in assemblies_colony_info_df_w_colonies_df.assembly_ID
                               for col in range(1,no_of_col_tested_per_sample+1)]
# create list of assemblies*dilutions
column_chunked_single_plating_list = list(divide_list_into_chunks(complete_cPCR_list_for_cPCR, well_avail_per_plate))
# allocate resupended colonies and cPCRs to plates
for plate_idx in range(0,len(resusp_colony_plate_objs)):
    # resuspended colonies
    extend_fill_plate_df_with_list(resusp_colony_plate_objs[plate_idx].layout, column_chunked_single_plating_list[plate_idx],inplace=True)
    # cPCR
    extend_fill_plate_df_with_list(cPCR_plate_objs[plate_idx].layout, column_chunked_single_plating_list[plate_idx],inplace=True)

72 out of 72 colonies are selected for cPCR screening, and 3 cPCRs will be prepared for each of them.
  - total cPCRs being performed: 216
  - wells_avail_per_plate: 96
  - no_required_plates: 3


In [17]:
print(f"Overview of used deck positions\n")
for plate_idx in range(0,len(resusp_colony_plate_objs)):
    print(f"Plating Round: {plate_idx}")
    resusp_colony_plate_styler = resusp_colony_plate_objs[plate_idx].layout.style.set_table_attributes("style='display:inline'").set_caption(f"Layout of {resusp_colony_plate_objs[plate_idx].name} ({resusp_colony_plate.parent})")
    cPCR_plate_styler = cPCR_plate_objs[plate_idx].layout.style.set_table_attributes("style='display:inline'").set_caption(f"Layout of {cPCR_plate_objs[plate_idx].name} ({cPCR_plate.parent})")
    space = "\xa0"*5
    display_html(resusp_colony_plate_styler._repr_html_()+space+cPCR_plate_styler._repr_html_(), raw=True)
    print(f"\n{'-'*100}\n")

Overview of used deck positions

Plating Round: 0


,1,2,3,4,5,6,7,8,9,10,11,12
A,pCMCW37_1,pCMCW39_3,pCMCW42_2,pCMCW45_1,pCMCW47_3,pCMCW50_2,pCMCW53_1,pCMCW55_3,pCMCW58_2,pCMCW61_1,pCMCW63_3,pCMCW66_2
B,pCMCW37_2,pCMCW40_1,pCMCW42_3,pCMCW45_2,pCMCW48_1,pCMCW50_3,pCMCW53_2,pCMCW56_1,pCMCW58_3,pCMCW61_2,pCMCW64_1,pCMCW66_3
C,pCMCW37_3,pCMCW40_2,pCMCW43_1,pCMCW45_3,pCMCW48_2,pCMCW51_1,pCMCW53_3,pCMCW56_2,pCMCW59_1,pCMCW61_3,pCMCW64_2,pCMCW67_1
D,pCMCW38_1,pCMCW40_3,pCMCW43_2,pCMCW46_1,pCMCW48_3,pCMCW51_2,pCMCW54_1,pCMCW56_3,pCMCW59_2,pCMCW62_1,pCMCW64_3,pCMCW67_2
E,pCMCW38_2,pCMCW41_1,pCMCW43_3,pCMCW46_2,pCMCW49_1,pCMCW51_3,pCMCW54_2,pCMCW57_1,pCMCW59_3,pCMCW62_2,pCMCW65_1,pCMCW67_3
F,pCMCW38_3,pCMCW41_2,pCMCW44_1,pCMCW46_3,pCMCW49_2,pCMCW52_1,pCMCW54_3,pCMCW57_2,pCMCW60_1,pCMCW62_3,pCMCW65_2,pCMCW68_1
G,pCMCW39_1,pCMCW41_3,pCMCW44_2,pCMCW47_1,pCMCW49_3,pCMCW52_2,pCMCW55_1,pCMCW57_3,pCMCW60_2,pCMCW63_1,pCMCW65_3,pCMCW68_2
H,pCMCW39_2,pCMCW42_1,pCMCW44_3,pCMCW47_2,pCMCW50_1,pCMCW52_3,pCMCW55_2,pCMCW58_1,pCMCW60_3,pCMCW63_2,pCMCW66_1,pCMCW68_3
,1,2,3,4,5,6,7,8,9,10,11,12
A,pCMCW37_1,pCMCW39_3,pCMCW42_2,pCMCW45_1,pCMCW47_3,pCMCW50_2,pCMCW53_1,pCMCW55_3,pCMCW58_2,pCMCW61_1,pCMCW63_3,pCMCW66_2



----------------------------------------------------------------------------------------------------

Plating Round: 1


,1,2,3,4,5,6,7,8,9,10,11,12
A,pCMCW69_1,pCMCW71_3,pCMCW74_2,pCMCW77_1,pCMCW79_3,pCMCW82_2,pCMCW85_1,pCMCW87_3,pCMCW90_2,pCMCW93_1,pCMCW95_3,pCMCW98_2
B,pCMCW69_2,pCMCW72_1,pCMCW74_3,pCMCW77_2,pCMCW80_1,pCMCW82_3,pCMCW85_2,pCMCW88_1,pCMCW90_3,pCMCW93_2,pCMCW96_1,pCMCW98_3
C,pCMCW69_3,pCMCW72_2,pCMCW75_1,pCMCW77_3,pCMCW80_2,pCMCW83_1,pCMCW85_3,pCMCW88_2,pCMCW91_1,pCMCW93_3,pCMCW96_2,pCMCW99_1
D,pCMCW70_1,pCMCW72_3,pCMCW75_2,pCMCW78_1,pCMCW80_3,pCMCW83_2,pCMCW86_1,pCMCW88_3,pCMCW91_2,pCMCW94_1,pCMCW96_3,pCMCW99_2
E,pCMCW70_2,pCMCW73_1,pCMCW75_3,pCMCW78_2,pCMCW81_1,pCMCW83_3,pCMCW86_2,pCMCW89_1,pCMCW91_3,pCMCW94_2,pCMCW97_1,pCMCW99_3
F,pCMCW70_3,pCMCW73_2,pCMCW76_1,pCMCW78_3,pCMCW81_2,pCMCW84_1,pCMCW86_3,pCMCW89_2,pCMCW92_1,pCMCW94_3,pCMCW97_2,pCMCW100_1
G,pCMCW71_1,pCMCW73_3,pCMCW76_2,pCMCW79_1,pCMCW81_3,pCMCW84_2,pCMCW87_1,pCMCW89_3,pCMCW92_2,pCMCW95_1,pCMCW97_3,pCMCW100_2
H,pCMCW71_2,pCMCW74_1,pCMCW76_3,pCMCW79_2,pCMCW82_1,pCMCW84_3,pCMCW87_2,pCMCW90_1,pCMCW92_3,pCMCW95_2,pCMCW98_1,pCMCW100_3
,1,2,3,4,5,6,7,8,9,10,11,12
A,pCMCW69_1,pCMCW71_3,pCMCW74_2,pCMCW77_1,pCMCW79_3,pCMCW82_2,pCMCW85_1,pCMCW87_3,pCMCW90_2,pCMCW93_1,pCMCW95_3,pCMCW98_2



----------------------------------------------------------------------------------------------------

Plating Round: 2


,1,2,3,4,5,6,7,8,9,10,11,12
A,pCMCW101_1,pCMCW103_3,pCMCW106_2,0,0,0,0,0,0,0,0,0
B,pCMCW101_2,pCMCW104_1,pCMCW106_3,0,0,0,0,0,0,0,0,0
C,pCMCW101_3,pCMCW104_2,pCMCW107_1,0,0,0,0,0,0,0,0,0
D,pCMCW102_1,pCMCW104_3,pCMCW107_2,0,0,0,0,0,0,0,0,0
E,pCMCW102_2,pCMCW105_1,pCMCW107_3,0,0,0,0,0,0,0,0,0
F,pCMCW102_3,pCMCW105_2,pCMCW108_1,0,0,0,0,0,0,0,0,0
G,pCMCW103_1,pCMCW105_3,pCMCW108_2,0,0,0,0,0,0,0,0,0
H,pCMCW103_2,pCMCW106_1,pCMCW108_3,0,0,0,0,0,0,0,0,0
,1,2,3,4,5,6,7,8,9,10,11,12
A,pCMCW101_1,pCMCW103_3,pCMCW106_2,0,0,0,0,0,0,0,0,0



----------------------------------------------------------------------------------------------------



In [18]:
assemblies_colony_info_df

,assembly_ID,assembly_description,Part 1: Promoter,Part 2: RBS,Part 3: CDS,Part 4: Terminator,Part 5: LVL1 Destination Vector,inter_stock_ID,assembly_tube,assembly_date,...,SOC_vol,CP_plate_format,CP_plate_no,CP_pos_per_round,CP_wells,colonies,cPCR_no1,cPCR_no2,cPCR_no3,sequence_verified
0,pCMCW37,CIDAR_LVL0_P_J23100_AB|CIDAR_LVL0_RBS_RiboJ-B0...,CIDAR_LVL0_P_J23100_AB,CIDAR_LVL0_RBS_RiboJ-B0032_BC,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_pSC101_sTU,CIDAR_LVL0_RBS_RiboJ-B0032_BC|CIDAR_LVL0_CDS_m...,A1,09/05/2022,...,125,corning_6_wellplate_16.8ml_flat,0,0,['A3'],Y,NaN,NaN,NaN,NaN
1,pCMCW38,CIDAR_LVL0_P_J23100_AB|CIDAR_LVL0_RBS_RiboJ-B0...,CIDAR_LVL0_P_J23100_AB,CIDAR_LVL0_RBS_RiboJ-B0032_BC,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_p15A_sTU,CIDAR_LVL0_RBS_RiboJ-B0032_BC|CIDAR_LVL0_CDS_m...,B1,09/05/2022,...,125,corning_6_wellplate_16.8ml_flat,0,0,['A2'],Y,NaN,NaN,NaN,NaN
2,pCMCW39,CIDAR_LVL0_P_J23100_AB|CIDAR_LVL0_RBS_RiboJ-B0...,CIDAR_LVL0_P_J23100_AB,CIDAR_LVL0_RBS_RiboJ-B0033,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_pSC101_sTU,CIDAR_LVL0_RBS_RiboJ-B0033|CIDAR_LVL0_CDS_mVen...,C1,09/05/2022,...,125,corning_6_wellplate_16.8ml_flat,0,0,['A1'],Y,NaN,NaN,NaN,NaN
3,pCMCW40,CIDAR_LVL0_P_J23100_AB|CIDAR_LVL0_RBS_RiboJ-B0...,CIDAR_LVL0_P_J23100_AB,CIDAR_LVL0_RBS_RiboJ-B0033,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_p15A_sTU,CIDAR_LVL0_RBS_RiboJ-B0033|CIDAR_LVL0_CDS_mVen...,D1,09/05/2022,...,125,corning_6_wellplate_16.8ml_flat,0,0,['B3'],Y,NaN,NaN,NaN,NaN
4,pCMCW41,CIDAR_LVL0_P_J23100_AB|CIDAR_LVL0_RBS_RiboJ-B0...,CIDAR_LVL0_P_J23100_AB,CIDAR_LVL0_RBS_RiboJ-B0034,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_pSC101_sTU,CIDAR_LVL0_RBS_RiboJ-B0034|CIDAR_LVL0_CDS_mVen...,E1,09/05/2022,...,125,corning_6_wellplate_16.8ml_flat,0,0,['B2'],Y,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,pCMCW104,CIDAR_LVL0_P_J23118|CIDAR_LVL0_RBS_RiboJ-B0032...,CIDAR_LVL0_P_J23118,CIDAR_LVL0_RBS_RiboJ-B0032_BC,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_p15A_sTU,CIDAR_LVL0_RBS_RiboJ-B0032_BC|CIDAR_LVL0_CDS_m...,B12,09/05/2022,...,125,corning_6_wellplate_16.8ml_flat,11,1,['A2'],Y,NaN,NaN,NaN,NaN
68,pCMCW105,CIDAR_LVL0_P_J23118|CIDAR_LVL0_RBS_RiboJ-B0033...,CIDAR_LVL0_P_J23118,CIDAR_LVL0_RBS_RiboJ-B0033,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_pSC101_sTU,CIDAR_LVL0_RBS_RiboJ-B0033|CIDAR_LVL0_CDS_mVen...,C12,09/05/2022,...,125,corning_6_wellplate_16.8ml_flat,11,1,['A1'],Y,NaN,NaN,NaN,NaN
69,pCMCW106,CIDAR_LVL0_P_J23118|CIDAR_LVL0_RBS_RiboJ-B0033...,CIDAR_LVL0_P_J23118,CIDAR_LVL0_RBS_RiboJ-B0033,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_p15A_sTU,CIDAR_LVL0_RBS_RiboJ-B0033|CIDAR_LVL0_CDS_mVen...,D12,09/05/2022,...,125,corning_6_wellplate_16.8ml_flat,11,1,['B3'],Y,NaN,NaN,NaN,NaN
70,pCMCW107,CIDAR_LVL0_P_J23118|CIDAR_LVL0_RBS_RiboJ-B0034...,CIDAR_LVL0_P_J23118,CIDAR_LVL0_RBS_RiboJ-B0034,CIDAR_LVL0_CDS_mVenus,CIDAR_LVL0_T_ECK120029600_DE,CIDAR_LVL0_LVL1-DVK_pSC101_sTU,CIDAR_LVL0_RBS_RiboJ-B0034|CIDAR_LVL0_CDS_mVen...,E12,09/05/2022,...,125,corning_6_wellplate_16.8ml_flat,11,1,['B2'],Y,NaN,NaN,NaN,NaN


## MM Preparation

In [19]:
MM_tube = large_vol_rack[reagent_pos_dic['MM']]
dead_vol = 25 # units: µl
assembly_surplus = 10 # in %
reagent_definitions_df['1x_prep_vol'] = round(reagent_definitions_df['1x_work_vol']*(assembly_surplus/100+1),2) 
prep_col_name, dead_vol_comp_factor = str(len(complete_cPCR_list_for_cPCR))+'x_prep_vol', dead_vol/reagent_definitions_df['1x_work_vol'].sum()
reagent_definitions_df[prep_col_name] = round(reagent_definitions_df['1x_prep_vol']*len(complete_cPCR_list_for_cPCR),2)
reagent_definitions_df[prep_col_name] = round(reagent_definitions_df[prep_col_name]+reagent_definitions_df['1x_work_vol']*dead_vol_comp_factor,2)
MM_vol = float(reagent_definitions_df['1x_work_vol'].sum()) # units: µl
print(f"\nA master mix of {round(reagent_definitions_df[prep_col_name].sum(),2)} µl in volume has to be prepared and put into position {MM_tube}.\n{MM_vol} µl will then be distributed and awaits resuspended colonies.\n")
display(reagent_definitions_df);


A master mix of 4107.4 µl in volume has to be prepared and put into position B3 of Opentrons 6 Tube Rack with Falcon 50 mL Conical on 4.
18.0 µl will then be distributed and awaits resuspended colonies.



,reagent,1x_work_vol,1x_prep_vol,216x_prep_vol
0,water1,6,6.30,1369.13
1,fw_primer,1,1.05,228.19
2,rv_primer,1,1.05,228.19
3,TF_DreamTaq_MM,10,10.50,2281.89


## Summary Spreadsheets

In [20]:
update_assembly_summary_answer = input("Do you want to update the GGA summary spreadsheet and save the cPCR-MM prep info? (yes / no)\n answer:")
if update_assembly_summary_answer == 'yes':
    assemblies_colony_info_df.to_csv(assemblies_info_dir,index=False)
    reagent_definitions_df.to_csv(PCR_reagent_definition_dir,index=False)
    print(f"\n -> Your GGA summary spreadsheet for experiment \'{experiment_name}\' has been updated!\n")
else:
    print(f"\n -> Ensure proper recording of your results!\n")
current_date = str(date.today())
current_date_mini = ''.join(current_date[2:].split('-'))
try:
    # create a Pandas Excel writer using openpyxl as the engine
    writer = pd.ExcelWriter(f"{current_date_mini}_{experiment_name}_plates_info.xlsx", engine='openpyxl')
    # write each dataframe to a different worksheet
    [resusp_colony_plate_objs[idx].layout.to_excel(writer, sheet_name=f"{resusp_colony_plate_objs[idx].name}") 
     for idx in range(0,len(resusp_colony_plate_objs))]
    [cPCR_plate_objs[idx].layout.to_excel(writer, sheet_name=f"{cPCR_plate_objs[idx].name}") 
     for idx in range(0,len(cPCR_plate_objs))]
    # close the Pandas Excel writer and output the Excel file
    writer.save()
    print(f" -> Maps of the PBS-resuspended colony plates and the cPCRs have been stored in {current_date_mini}_{experiment_name}_plates_info.xlsx\n")
except:
    pass

Do you want to update the GGA summary spreadsheet and save the cPCR-MM prep info? (yes / no)
 answer: yes



 -> Your GGA summary spreadsheet for experiment 'sTUFPlibrary' has been updated!

 -> Maps of the PBS-resuspended colony plates and the cPCRs have been stored in 220515_sTUFPlibrary_plates_info.xlsx



In [21]:
temp_mod.set_temperature(6)
if audiofeedback == True:
    allDone()

---
# 6- Execution


## i. PBS distribution

In [22]:
PBS_resuspension_vol = 25 # units: µl
PBS_tube = large_vol_rack[reagent_pos_dic['PBS']]
pbs_start_time = time.time()
for plate_idx, pbs_plate in enumerate(resusp_colony_plate_objs):
    print(f"\nPBS Distribution round {plate_idx} / {len(resusp_colony_plate_objs)-1}")
    resusp_col_wellname_list = list(pbs_plate.occupied_wells.keys())
    # exchange PBS-resuspension plate whenever one round of plating fills up all available wells
    plate_load_question = 'no'
    while plate_load_question != 'yes':
        plate_load_question = input(f"\nHave you loaded the {resusp_colony_plate_objs[plate_idx].name} in position {resusp_colony_plate.parent} ? (yes/no) \n  answer:")
        if plate_load_question == 'skip':
            print(f"WARNING: {resusp_colony_plate_objs[plate_idx].name} has been skipped!")
            break
    if plate_load_question != 'skip':
        # update user on liquid handling status
        print(f"\n  - {PBS_resuspension_vol} µl PBS ({PBS_resuspension_vol*len(resusp_col_wellname_list)} µl needed) from {PBS_tube}\n        -> {resusp_colony_plate_objs[plate_idx].name}:\n")
        pprint(list(divide_list_into_chunks(resusp_col_wellname_list,8)))
        smart_distribute_liquid(PBS_resuspension_vol,
                                PBS_tube, [resusp_colony_plate.wells_by_name()[well_name] for well_name in resusp_col_wellname_list],
                                asp_bot_clearance=2.,dispens_bot_clearance=3,
                                touch_tip_bool=False,new_disposal_volume=0
                               )
    print(f"\n{'-'*100}\n")
pbs_end_time = time.time()
print(f"(PBS distribution took {show_time(pbs_end_time-pbs_start_time)})\n\n Now pick colonies and resuspend them in the correct ResCol_Plate wells!")
if audiofeedback == True:
    allDone()


PBS Distribution round 0 / 2



Have you loaded the ResCol_Plate_0 in position 2 ? (yes/no) 
  answer: yes



  - 25 µl PBS (2400 µl needed) from A3 of Opentrons 6 Tube Rack with Falcon 50 mL Conical on 4
        -> ResCol_Plate_0:

[['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1'],
 ['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2'],
 ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3'],
 ['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4'],
 ['A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5'],
 ['A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6'],
 ['A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7'],
 ['A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8'],
 ['A9', 'B9', 'C9', 'D9', 'E9', 'F9', 'G9', 'H9'],
 ['A10', 'B10', 'C10', 'D10', 'E10', 'F10', 'G10', 'H10'],
 ['A11', 'B11', 'C11', 'D11', 'E11', 'F11', 'G11', 'H11'],
 ['A12', 'B12', 'C12', 'D12', 'E12', 'F12', 'G12', 'H12']]

----------------------------------------------------------------------------------------------------


PBS Distribution round 1 / 2



Have you loaded the ResCol_Plate_1 in position 2 ? (yes/no) 
  answer: yes



  - 25 µl PBS (2400 µl needed) from A3 of Opentrons 6 Tube Rack with Falcon 50 mL Conical on 4
        -> ResCol_Plate_1:

[['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1'],
 ['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2'],
 ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3'],
 ['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4'],
 ['A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5'],
 ['A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6'],
 ['A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7'],
 ['A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8'],
 ['A9', 'B9', 'C9', 'D9', 'E9', 'F9', 'G9', 'H9'],
 ['A10', 'B10', 'C10', 'D10', 'E10', 'F10', 'G10', 'H10'],
 ['A11', 'B11', 'C11', 'D11', 'E11', 'F11', 'G11', 'H11'],
 ['A12', 'B12', 'C12', 'D12', 'E12', 'F12', 'G12', 'H12']]

----------------------------------------------------------------------------------------------------


PBS Distribution round 2 / 2



Have you loaded the ResCol_Plate_2 in position 2 ? (yes/no) 
  answer: yes



  - 25 µl PBS (600 µl needed) from A3 of Opentrons 6 Tube Rack with Falcon 50 mL Conical on 4
        -> ResCol_Plate_2:

[['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1'],
 ['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2'],
 ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3']]

----------------------------------------------------------------------------------------------------

(PBS distribution took 00:00:10)

 Now pick colonies and resuspend them in the correct ResCol_Plate wells!


## ii. cPCR-MM Prep

In [23]:
if not any(reagent_definitions_df[prep_col_name]>300):
    # go through reagent information and assemble master mix in reagent rack
    total_vol_in_target = 0
    print(f"Preparation of cPCR master mix for {no_of_col_tested_per_sample} colonies * {len(assemblies_colony_info_df_w_colonies_df)} assemblies ({len(complete_cPCR_list_for_cPCR)} cPCRs):")
    for idx, reagent_info in enumerate([reagent_definitions_df.iloc[row_no,:] for row_no in range(len(reagent_definitions_df))]):
        source_location = reagents_rack[find_df_coordinates(reagents_rack_obj.layout,reagent_info.reagent)]
        transfer_vol = reagent_info[prep_col_name]
        total_vol_in_target += transfer_vol
        print(f" - {reagent_info.reagent}:\n    reagent rack: {transfer_vol} µl \'{source_location.well_name}\' -> master mix \'{MM_tube.well_name}\'")
        # perform cyclical mixing only after last reagent has been added
        if idx == len([reagent_definitions_df.iloc[row_no,:] for row_no in range(len(reagent_definitions_df))]):
            smart_transfer_liquid(transfer_vol, source_location, MM_tube,
                                  asp_bot_clearance=1.2,asp_rate=0.4, touch_tip_before=False,touch_tip=False,
                                  mix_before_reps=1,cyclical_mix_after=(2, transfer_vol, idx, calc_eppi_height(total_vol_in_target)-2),
                                  dispens_bot_clearance=2.0,disp_rate=1.1)
        else:
            smart_transfer_liquid(transfer_vol, source_location, MM_tube,
                                  asp_bot_clearance=1.2,asp_rate=0.4, touch_tip_before=False,touch_tip=False,
                                  mix_before_reps=1,
                                  dispens_bot_clearance=2.0,disp_rate=1.1)
else:
    print(f"WARNING: Reagent volumes are too high to automatically pipette!\n -> Create the MM manually and add it to {MM_tube}")

 -> Create the MM manually and add it to B3 of Opentrons 6 Tube Rack with Falcon 50 mL Conical on 4


## iii. cPCR-MM distribution

In [24]:
cPCR_start_time = time.time()
for plate_idx, current_cPCR_plate in enumerate(cPCR_plate_objs):
    print(f"\ncPCR-MM Distribution round {plate_idx} / {len(resusp_colony_plate_objs)-1}")
    cPCR_wellname_list = list(current_cPCR_plate.occupied_wells.keys())
    # exchange PBS-resuspension plate whenever one round of plating fills up all available wells
    plate_load_question = 'no'
    while plate_load_question != 'yes':
        plate_load_question = input(f"\nHave you loaded the {cPCR_plate_objs[plate_idx].name} in position(s) {cPCR_plate.parent} ? (yes/no) \n  answer:")
        if plate_load_question == 'skip':
            print(f"WARNING: {cPCR_plate_objs[plate_idx].name} has been skipped!")
            break
    if plate_load_question != 'skip':
        # update user on liquid handling status
        print(f"\n  - {MM_vol} µl cPCR-MM ({MM_vol*len(cPCR_wellname_list)} µl needed) from {MM_tube}\n        -> {cPCR_plate_objs[plate_idx].name}:\n")
        pprint(list(divide_list_into_chunks(cPCR_wellname_list,8)))
        
        smart_distribute_liquid(MM_vol,
                                MM_tube, 
                                [cPCR_plate.wells_by_name()[well_name] for well_name in cPCR_wellname_list],
                                asp_bot_clearance=1.2,dispens_bot_clearance=2,
                                touch_tip_bool=False,new_disposal_volume=0
                               )
    print(f"\n{'-'*100}\n")
cPCR_end_time = time.time()
print(f"(PBS distribution took {show_time(cPCR_end_time-cPCR_start_time)})\n")
if audiofeedback == True:
    allDone()


cPCR-MM Distribution round 0 / 2



Have you loaded the cPCR_Plate_0 in position(s) Temperature Module GEN2 on 3 ? (yes/no) 
  answer: yes



  - 18.0 µl cPCR-MM (1728.0 µl needed) from B3 of Opentrons 6 Tube Rack with Falcon 50 mL Conical on 4
        -> cPCR_Plate_0:

[['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1'],
 ['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2'],
 ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3'],
 ['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4'],
 ['A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5'],
 ['A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6'],
 ['A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7'],
 ['A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8'],
 ['A9', 'B9', 'C9', 'D9', 'E9', 'F9', 'G9', 'H9'],
 ['A10', 'B10', 'C10', 'D10', 'E10', 'F10', 'G10', 'H10'],
 ['A11', 'B11', 'C11', 'D11', 'E11', 'F11', 'G11', 'H11'],
 ['A12', 'B12', 'C12', 'D12', 'E12', 'F12', 'G12', 'H12']]

----------------------------------------------------------------------------------------------------


cPCR-MM Distribution round 1 / 2



Have you loaded the cPCR_Plate_1 in position(s) Temperature Module GEN2 on 3 ? (yes/no) 
  answer: yes



  - 18.0 µl cPCR-MM (1728.0 µl needed) from B3 of Opentrons 6 Tube Rack with Falcon 50 mL Conical on 4
        -> cPCR_Plate_1:

[['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1'],
 ['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2'],
 ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3'],
 ['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4'],
 ['A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5'],
 ['A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6'],
 ['A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7'],
 ['A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8'],
 ['A9', 'B9', 'C9', 'D9', 'E9', 'F9', 'G9', 'H9'],
 ['A10', 'B10', 'C10', 'D10', 'E10', 'F10', 'G10', 'H10'],
 ['A11', 'B11', 'C11', 'D11', 'E11', 'F11', 'G11', 'H11'],
 ['A12', 'B12', 'C12', 'D12', 'E12', 'F12', 'G12', 'H12']]

----------------------------------------------------------------------------------------------------


cPCR-MM Distribution round 2 / 2



Have you loaded the cPCR_Plate_2 in position(s) Temperature Module GEN2 on 3 ? (yes/no) 
  answer: yes



  - 18.0 µl cPCR-MM (432.0 µl needed) from B3 of Opentrons 6 Tube Rack with Falcon 50 mL Conical on 4
        -> cPCR_Plate_2:

[['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1'],
 ['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2'],
 ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3']]

----------------------------------------------------------------------------------------------------

(PBS distribution took 00:00:09)



## iv. Colony Transfer

In [25]:
col_transfer_start_time = time.time()
counter = 1
for plate_idx, current_plates_resusp_colony_names in enumerate(column_chunked_single_plating_list):
    print(f"\nTransfer of resuspended colonies to cPCR-MM - round {plate_idx} / {len(column_chunked_single_plating_list)-1}")
    # exchange both the PBS-resuspended colony plate + the cPCR plate whenever one round of plating fills up all available wells
    replenish_tiprack(tiprack20a)
    plate_load_question = 'no'
    while plate_load_question != 'yes':
        plate_load_question = input(f"\nHave you loaded {resusp_colony_plate_objs[plate_idx].name} in position {resusp_colony_plate.parent} &\nhave you loaded   {cPCR_plate_objs[plate_idx].name} in position {cPCR_plate.parent} ? (yes/no) \n  answer:")
        print("\n")
        if plate_load_question == 'skip':
            print(f"WARNING: {cPCR_plate_objs[plate_idx].name} has been skipped!")
            break
    # loop through every part, identify its location on the PBS-resuspended colony plate + the cPCR plate & then execute transfer
    for idx, current_assembly_name in enumerate(current_plates_resusp_colony_names):
        source_location = resusp_colony_plate[find_df_coordinates(resusp_colony_plate_objs[plate_idx].layout,current_assembly_name)]
        target_location = cPCR_plate[find_df_coordinates(cPCR_plate_objs[plate_idx].layout,current_assembly_name)]
        if plate_load_question != 'skip':
            # update user on liquid handling status
            print(f"  {str(counter).zfill(2)}/{str(total_no_cPCRs).zfill(2)}- {resusp_col_for_cPCR_vol} µl {current_assembly_name}:\n              resuspended_colony_plate \'{source_location.well_name}\' -> cPCR_plate \'{target_location.well_name}\'")
            smart_transfer_liquid(resusp_col_for_cPCR_vol,
                                  source_location, target_location,
                                  asp_bot_clearance=1.2,asp_rate=0.8, touch_tip_before=False,touch_tip=False,
                                  blow_out=False,mix_before_reps=0,
                                  dispens_bot_clearance=2.2,disp_rate=0.9)
            counter+=1
    print(f"\n{'-'*100}\n")
col_transfer_end_time = time.time()
print(f"(Colony transfers took {show_time(col_transfer_end_time-col_transfer_start_time)})\n")
if audiofeedback == True:
    allDone()


Transfer of resuspended colonies to cPCR-MM - round 0 / 2



    [Have you replenished all the tips in Opentrons 96 Tip Rack 20 µL on 11? (yes/no)
      answer: yes


              Excellent! Now you can continue running the rest of the protocol.]



Have you loaded ResCol_Plate_0 in position 2 &
have you loaded   cPCR_Plate_0 in position Temperature Module GEN2 on 3 ? (yes/no) 
  answer: yes




  01/216- 2.0 µl pCMCW37_1:
              resuspended_colony_plate 'A1' -> cPCR_plate 'A1'
  02/216- 2.0 µl pCMCW37_2:
              resuspended_colony_plate 'B1' -> cPCR_plate 'B1'
  03/216- 2.0 µl pCMCW37_3:
              resuspended_colony_plate 'C1' -> cPCR_plate 'C1'
  04/216- 2.0 µl pCMCW38_1:
              resuspended_colony_plate 'D1' -> cPCR_plate 'D1'
  05/216- 2.0 µl pCMCW38_2:
              resuspended_colony_plate 'E1' -> cPCR_plate 'E1'
  06/216- 2.0 µl pCMCW38_3:
              resuspended_colony_plate 'F1' -> cPCR_plate 'F1'
  07/216- 2.0 µl pCMCW39_1:
              resuspended_colony_plate 'G1' -> cPCR_plate 'G1'
  08/216- 2.0 µl pCMCW39_2:
              resuspended_colony_plate 'H1' -> cPCR_plate 'H1'
  09/216- 2.0 µl pCMCW39_3:
              resuspended_colony_plate 'A2' -> cPCR_plate 'A2'
  10/216- 2.0 µl pCMCW40_1:
              resuspended_colony_plate 'B2' -> cPCR_plate 'B2'
  11/216- 2.0 µl pCMCW40_2:
              resuspended_colony_plate 'C2' -> cPCR_plate 'C

...



  97/216- 2.0 µl pCMCW69_1:
              resuspended_colony_plate 'A1' -> cPCR_plate 'A1'
  98/216- 2.0 µl pCMCW69_2:
              resuspended_colony_plate 'B1' -> cPCR_plate 'B1'
  99/216- 2.0 µl pCMCW69_3:
              resuspended_colony_plate 'C1' -> cPCR_plate 'C1'
  100/216- 2.0 µl pCMCW70_1:
              resuspended_colony_plate 'D1' -> cPCR_plate 'D1'
  101/216- 2.0 µl pCMCW70_2:
              resuspended_colony_plate 'E1' -> cPCR_plate 'E1'
  102/216- 2.0 µl pCMCW70_3:
              resuspended_colony_plate 'F1' -> cPCR_plate 'F1'
  103/216- 2.0 µl pCMCW71_1:
              resuspended_colony_plate 'G1' -> cPCR_plate 'G1'
  104/216- 2.0 µl pCMCW71_2:
              resuspended_colony_plate 'H1' -> cPCR_plate 'H1'
  105/216- 2.0 µl pCMCW71_3:
              resuspended_colony_plate 'A2' -> cPCR_plate 'A2'
  106/216- 2.0 µl pCMCW72_1:
              resuspended_colony_plate 'B2' -> cPCR_plate 'B2'
  107/216- 2.0 µl pCMCW72_2:
              resuspended_colony_plate 'C2' -> cPCR_


    [Have you replenished all the tips in Opentrons 96 Tip Rack 20 µL on 11? (yes/no)
      answer: yes


              Excellent! Now you can continue running the rest of the protocol.]



Have you loaded ResCol_Plate_2 in position 2 &
have you loaded   cPCR_Plate_2 in position Temperature Module GEN2 on 3 ? (yes/no) 
  answer: yes




  193/216- 2.0 µl pCMCW101_1:
              resuspended_colony_plate 'A1' -> cPCR_plate 'A1'
  194/216- 2.0 µl pCMCW101_2:
              resuspended_colony_plate 'B1' -> cPCR_plate 'B1'
  195/216- 2.0 µl pCMCW101_3:
              resuspended_colony_plate 'C1' -> cPCR_plate 'C1'
  196/216- 2.0 µl pCMCW102_1:
              resuspended_colony_plate 'D1' -> cPCR_plate 'D1'
  197/216- 2.0 µl pCMCW102_2:
              resuspended_colony_plate 'E1' -> cPCR_plate 'E1'
  198/216- 2.0 µl pCMCW102_3:
              resuspended_colony_plate 'F1' -> cPCR_plate 'F1'
  199/216- 2.0 µl pCMCW103_1:
              resuspended_colony_plate 'G1' -> cPCR_plate 'G1'
  200/216- 2.0 µl pCMCW103_2:
              resuspended_colony_plate 'H1' -> cPCR_plate 'H1'
  201/216- 2.0 µl pCMCW103_3:
              resuspended_colony_plate 'A2' -> cPCR_plate 'A2'
  202/216- 2.0 µl pCMCW104_1:
              resuspended_colony_plate 'B2' -> cPCR_plate 'B2'
  203/216- 2.0 µl pCMCW104_2:
              resuspended_colony_plate

# Shutdown

In [26]:
# allDone()
try:
    p20_left.drop_tip()
except:
    pass
try:
    p300_right.drop_tip()
except:
    pass
temp_mod.deactivate()
protocol.home()
#os.system("systemctl start opentrons-robot-server")

In [27]:
col_no = 25
col_that_can_be_run_on_gel_at_the_same_time = 4 
no_of_gel_runs = math.ceil(col_no / col_that_can_be_run_on_gel_at_the_same_time)
time_per_gel_run = 0.67 # unit: h
total_no_of_running_gels = no_of_gel_runs * time_per_gel_run * 1.1
print(f" - no_of_gel_runs: {no_of_gel_runs}\n - total_no_of_running_gels: {round(total_no_of_running_gels,2)} hours ")

 - no_of_gel_runs: 7
 - total_no_of_running_gels: 5.16 hours 


In [28]:
# complete tip usage check:
_=[display(show_rack_usage(labware)[1]) for labware in labware_list if 'Opentrons 96 Tip Rack' in str(labware)]

Opentrons 96 Tip Rack 20 µL on 11:
 -> 24 tips used:


,1,2,3,4,5,6,7,8,9,10,11,12
A,x,x,x,0,0,0,0,0,0,0,0,0
B,x,x,x,0,0,0,0,0,0,0,0,0
C,x,x,x,0,0,0,0,0,0,0,0,0
D,x,x,x,0,0,0,0,0,0,0,0,0
E,x,x,x,0,0,0,0,0,0,0,0,0
F,x,x,x,0,0,0,0,0,0,0,0,0
G,x,x,x,0,0,0,0,0,0,0,0,0
H,x,x,x,0,0,0,0,0,0,0,0,0


Opentrons 96 Tip Rack 20 µL on 10:
 -> 0 tips used:


,1,2,3,4,5,6,7,8,9,10,11,12
A,0,0,0,0,0,0,0,0,0,0,0,0
B,0,0,0,0,0,0,0,0,0,0,0,0
C,0,0,0,0,0,0,0,0,0,0,0,0
D,0,0,0,0,0,0,0,0,0,0,0,0
E,0,0,0,0,0,0,0,0,0,0,0,0
F,0,0,0,0,0,0,0,0,0,0,0,0
G,0,0,0,0,0,0,0,0,0,0,0,0
H,0,0,0,0,0,0,0,0,0,0,0,0


Opentrons 96 Tip Rack 300 µL on 7:
 -> 4 tips used:


,1,2,3,4,5,6,7,8,9,10,11,12
A,x,0,0,0,0,0,0,0,0,0,0,0
B,x,0,0,0,0,0,0,0,0,0,0,0
C,x,0,0,0,0,0,0,0,0,0,0,0
D,x,0,0,0,0,0,0,0,0,0,0,0
E,0,0,0,0,0,0,0,0,0,0,0,0
F,0,0,0,0,0,0,0,0,0,0,0,0
G,0,0,0,0,0,0,0,0,0,0,0,0
H,0,0,0,0,0,0,0,0,0,0,0,0
